In [1]:
#import necessary libraries

from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import os
import glob

from PIL import Image
from io import BytesIO

import matplotlib.pyplot as plt
from IPython.display import display

import plotly.graph_objects as go
import plotly.io as pio

import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from scipy.signal import correlate2d
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array

from scipy.signal import find_peaks
import math

import pandas as pd
from tqdm import tqdm

import warnings 
warnings.filterwarnings('ignore')


In [2]:
def get_exif_data(image_path):
    """
    Function to extract specific EXIF metadata - GPS coordinates, 
    camera model, and focal length from an image file. 
    --------------------------------------------------------------
    Parameters:
    image_path: A string representing the file path of the image from which the EXIF data is to be extracted.
    --------------------------------------------------------------
    Returns:
    A tuple of lists containing the EXIF tags and their corresponding values.
    """
    desired_tags = ['GPSLatitudeRef', 'GPSLatitude', 'GPSLongitudeRef', 'GPSLongitude', 'GPSAltitude', 'Model', 'FocalLength', "DateTime"]

    image = Image.open(image_path)
    exif_data = image._getexif()
    exif_dict = {}

    if exif_data is not None:
        for key, value in exif_data.items():
            if key in TAGS:
                tag = TAGS[key]

                if tag == 'GPSInfo':
                    gps_data = {}
                    for t in value:
                        sub_tag = GPSTAGS.get(t, t)
                        if sub_tag in desired_tags:
                            gps_data[sub_tag] = value[t]
                    exif_dict.update(gps_data)
                elif tag in desired_tags:
                    exif_dict[tag] = value

    return exif_dict


def get_geo_coord(lat, ref_lat, lon, ref_lon):
    """
    Function to convert EXIF GPS coordinates to decimal format.
    --------------------------------------------------------------
    Parameters:
    lat: A list of tuples containing the GPS latitude coordinates in degrees, minutes, and seconds.
    ref_lat: A string representing the reference direction(N, S) of the GPS latitude coordinates.
    lon: A list of tuples containing the GPS longitude coordinates in degrees, minutes, and seconds.
    ref_lon: A string representing the reference direction(E, W) of the GPS longitude coordinates.
    --------------------------------------------------------------
    Returns:
    A tuple of floats representing the GPS coordinates in decimal format.
    """
    deg, minutes, seconds = lat
    decimal_deg_lat = deg + (minutes / 60.0) + (seconds / 3600.0)

    # Adjusting for the reference direction
    if ref_lat == 'S':
        decimal_deg_lat *= -1

    deg, minutes, seconds = lon
    decimal_deg_lon = deg + (minutes / 60.0) + (seconds / 3600.0)

    # Adjusting for the reference direction
    if ref_lon == 'W':
        decimal_deg_lon *= -1

    return decimal_deg_lat, decimal_deg_lon


def fetch_map_image(lat, lon, zoom, size, mapbox_access_token):
    """
    Function to fetch a static map image using Plotly.
    --------------------------------------------------------------
    Parameters:
    lat (float): Latitude of the location.
    lon (float): Longitude of the location.
    zoom (int): Zoom level of the image.
    size (str): Size of the image in 'widthxheight' format (e.g., '600x400').
    altitude (float, optional): Altitude of the location. Used for marker representation.
    mapbox_access_token (str): Mapbox access token for rendering the map.
    --------------------------------------------------------------
    Returns:
    A static image (PNG) of the location.
    """
    width, height = map(int, size.split('x'))

    # Create a scatter mapbox plot with Plotly
    fig = go.Figure(go.Scattermapbox(
        lat=[lat],
        lon=[lon],
    ))

    # Set the layout of the map
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token,
            center=go.layout.mapbox.Center(lat=lat, lon=lon),
            zoom=zoom,
            style='satellite'
        ),
        width=width,
        height=height,
        margin=dict(l=0, r=0, t=0, b=0),
    )

    # Generate the figure as a PNG byte array
    image_data = pio.to_image(fig, format='png')

    pil_image = Image.open(BytesIO(image_data))

    return pil_image

def extract_features(img, model):
    """
    Function to preprocess the image and extract features
    --------------------------------------------------------------
    Parameters:
    img: A PIL image object.
    model: A Keras model object.
    --------------------------------------------------------------
    Returns:
    A 4D NumPy array containing the extracted features.
    """
    img_array = img_to_array(img.resize((512, 512)))
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = model.predict(img_array)
    return features


def find_query_in_aoi(query_features, aoi_features):
    """
    Author: ChatGPT-4
    Function to find the location of the query image in the AOI image
    --------------------------------------------------------------
    Parameters:
    query_features: A 4D NumPy array containing the features of the query image.
    aoi_features: A 4D NumPy array containing the features of the AOI image.
    --------------------------------------------------------------
    Returns:
    A tuple containing the x and y coordinates of the query image in the AOI image.
    """
    # Perform cross-correlation between the features of the query image and the AOI image
    correlation_map = correlate2d(aoi_features[0, :, :, 0], query_features[0, :, :, 0], boundary='wrap', mode='same')
    
    # Find the location with the highest correlation
    y, x = np.unravel_index(np.argmax(correlation_map), correlation_map.shape)
    
    return (x, y), correlation_map[y, x], correlation_map


def mercator_projection(lat):
    """
    Author: ChatGPT-4
    This function converts latitude to Mercator projection.
    --------------------------------------------------------------
    Parameters:
    lat (float): Latitude in degrees.
    --------------------------------------------------------------
    Returns:
    float: Latitude in Mercator projection.
    """
    return math.log(math.tan(math.radians(lat) / 2 + math.pi / 4))

def calculate_mapbox_bounding_box(lat_center, lon_center, zoom, image_width, image_height):
    """
    Author: ChatGPT-4
    This function calculates the bounding box of a Mapbox map.
    --------------------------------------------------------------
    Parameters:
    lat_center (float): Latitude of the center of the map.
    lon_center (float): Longitude of the center of the map.
    zoom (int): Zoom level of the map.
    image_width (int): Width of the image.
    image_height (int): Height of the image.
    --------------------------------------------------------------
    Returns:
    tuple: A tuple containing the coordinates of the top-left, top-right, bottom-right, and bottom-left corners of the image.
    """
    # Tile size (in pixels) used by Mapbox
    tile_size = 512

    # Number of tiles at the given zoom level
    num_tiles = 2 ** zoom

    # Scale factor at this zoom level
    scale = num_tiles * tile_size

    # Latitude in Mercator projection
    mercator_lat = mercator_projection(lat_center)

    # Convert center longitude and latitude to pixel values
    pixel_x_center = (lon_center + 180) / 360 * scale
    pixel_y_center = (1 - mercator_lat / math.pi) / 2 * scale

    # Calculate pixel coordinates of the corners
    pixel_x_left = pixel_x_center - image_width / 2
    pixel_x_right = pixel_x_center + image_width / 2
    pixel_y_top = pixel_y_center - image_height / 2
    pixel_y_bottom = pixel_y_center + image_height / 2

    # Convert pixel coordinates back to lat/lon
    def pixels_to_latlon(px, py):
        lon = px / scale * 360 - 180
        lat = math.degrees(2 * math.atan(math.exp((1 - 2 * py / scale) * math.pi)) - math.pi / 2)
        return lat, lon

    top_left = pixels_to_latlon(pixel_x_left, pixel_y_top)
    top_right = pixels_to_latlon(pixel_x_right, pixel_y_top)
    bottom_right = pixels_to_latlon(pixel_x_right, pixel_y_bottom)
    bottom_left = pixels_to_latlon(pixel_x_left, pixel_y_bottom)

    return top_left, top_right, bottom_right, bottom_left

def find_best_matches(correlation_map):
    """
    This function finds the best and second-best match in the correlation map.
    --------------------------------------------------------------
    Parameters:
    correlation_map (numpy.ndarray): The correlation map obtained from comparing the query image and the AOI.
    --------------------------------------------------------------
    Returns:
    tuple: A tuple containing the coordinates of the best and second-best matches.
    """
    correlation_map_flat = correlation_map.flatten()
    peaks, _ = find_peaks(correlation_map_flat, height=0)

    if len(peaks) == 0:
        return None, None
    
    elif len(peaks) == 1:
        return peaks, None

    else:
        sorted_peaks = sorted(peaks, key=lambda x: correlation_map_flat[x], reverse=True)
        best_match_index = sorted_peaks[0]
        second_best_match_index = sorted_peaks[1]
        best_match_coords = np.unravel_index(best_match_index, correlation_map.shape)
        second_best_match_coords = np.unravel_index(second_best_match_index, correlation_map.shape)
        return best_match_coords, second_best_match_coords

def find_midpoint(coord1, coord2):
    """
    This function calculates the midpoint between two coordinates.
    --------------------------------------------------------------
    Parameters:
    coord1 (tuple): The first coordinate (x1, y1).
    coord2 (tuple): The second coordinate (x2, y2).
    --------------------------------------------------------------
    Returns:
    tuple: The midpoint coordinates.
    """
    midpoint = ((coord1[0] + coord2[0]) / 2, (coord1[1] + coord2[1]) / 2)
    return midpoint

def scale_correlation_to_aoi(midpoint, correlation_shape, aoi_size):
    """
    Scale the correlation map coordinates to the AOI image size.
    --------------------------------------------------------------
    Parameters:
    midpoint (tuple): The midpoint coordinates of the correlation map.
    correlation_shape (tuple): The shape of the correlation map.
    aoi_size (tuple): The size of the AOI image.
    --------------------------------------------------------------
    Returns:
    tuple: The scaled coordinates.
    """
    x_scale = aoi_size[0] / correlation_shape[1]
    y_scale = aoi_size[1] / correlation_shape[0]
    
    scaled_x = midpoint[0] * x_scale
    scaled_y = midpoint[1] * y_scale
    
    return scaled_x, scaled_y

def interpolate_geo_coordinates(aoi_midpoint, top_left, bottom_right, aoi_size):
    """
    Author: ChatGPT-4
    Interpolate the AOI image coordinates to geographical coordinates.
    --------------------------------------------------------------
    Parameters:
    aoi_midpoint (tuple): The midpoint coordinates of the AOI image.
    top_left (tuple): The top-left coordinates of the AOI image.
    bottom_right (tuple): The bottom-right coordinates of the AOI image.
    aoi_size (tuple): The size of the AOI image.
    --------------------------------------------------------------
    Returns:
    tuple: The interpolated geographical coordinates.
    """
    lat_range = top_left[0] - bottom_right[0]
    lon_range = bottom_right[1] - top_left[1]
    
    lat_per_pixel = lat_range / aoi_size[1]
    lon_per_pixel = lon_range / aoi_size[0]
    
    lat = top_left[0] - (aoi_midpoint[1] * lat_per_pixel)
    lon = top_left[1] + (aoi_midpoint[0] * lon_per_pixel)
    
    return lat, lon


In [3]:
mapbox_access_token = input('Enter Mapbox Aceess Token: ')
zoom_level = 9.5
image_size = '3000x3000' 
model = VGG16(weights='imagenet', include_top=False)

for f in tqdm(glob.glob('../data/images/*.jpg'), desc="Working on files"):
    df = pd.read_csv('../results/NN-Eval.csv')

    query_image_name = os.path.basename(f).split('.')[0]
    df_index = df[df['Image'] == query_image_name].index

    if (len(df_index) == 0):
        new_row = {'Image': query_image_name}
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
        df_index = df[df['Image'] == query_image_name].index

    query_img = Image.open(f)
    exif_data = get_exif_data(f)

    if df_index.any():
        if pd.isna(df.at[df_index[0], 'Best Match 1']) and pd.isna(df.at[df_index[0], 'Issues']):
            gps_latitude = exif_data.get('GPSLatitude', None)
            gps_latitude_ref = exif_data.get('GPSLatitudeRef', None)
            gps_longitude = exif_data.get('GPSLongitude', None)
            gps_longitude_ref = exif_data.get('GPSLongitudeRef', None)
            
            if gps_latitude and gps_latitude_ref and gps_longitude and gps_longitude_ref:
                lat, long = get_geo_coord(gps_latitude, gps_latitude_ref, gps_longitude, gps_longitude_ref)
                aoi_img = fetch_map_image(lat, long, zoom_level, image_size, mapbox_access_token)
                if aoi_img.mode != 'RGB':
                    aoi_img = aoi_img.convert('RGB')
                
                query_features = extract_features(query_img, model)
                aoi_features = extract_features(aoi_img, model)
                location, _, correlation_map = find_query_in_aoi(query_features, aoi_features)
                
                top_left_aoi, top_right_aoi, bottom_right_aoi, bottom_left_aoi = calculate_mapbox_bounding_box(lat, long, zoom_level, aoi_img.width, aoi_img.height)
                best_match, second_best_match = find_best_matches(correlation_map)

                if best_match is not None and second_best_match is not None:
                    aoi_best_match = scale_correlation_to_aoi(best_match, correlation_map.shape, (aoi_img.width, aoi_img.height))
                    geo_coords_best_match = interpolate_geo_coordinates(aoi_best_match, top_left_aoi, bottom_right_aoi, (aoi_img.width, aoi_img.height))

                    aoi_second_best_match = scale_correlation_to_aoi(second_best_match, correlation_map.shape, (aoi_img.width, aoi_img.height))
                    geo_coords_second_best_match = interpolate_geo_coordinates(aoi_second_best_match, top_left_aoi, bottom_right_aoi, (aoi_img.width, aoi_img.height))

                    midpoint = find_midpoint(best_match, second_best_match)
                    aoi_midpoint = scale_correlation_to_aoi(midpoint, correlation_map.shape, (aoi_img.width, aoi_img.height))
                    geo_coords = interpolate_geo_coordinates(aoi_midpoint, top_left_aoi, bottom_right_aoi, (aoi_img.width, aoi_img.height))
                    
                    if df_index.any():
                        df.at[df_index[0], 'Best Match 1'] = str(f'{geo_coords_best_match[0]}, {geo_coords_best_match[1]}')
                        df.at[df_index[0], 'Best Match 2'] = str(f'{geo_coords_second_best_match[0]}, {geo_coords_second_best_match[1]}')
                        df.at[df_index[0], 'Predicted Location'] = str(f'{geo_coords[0]}, {geo_coords[1]}')
                
                elif best_match is not None and second_best_match is None:
                    if df_index.any():
                        df.at[df_index[0], 'Best Match 1'] = str(f'{geo_coords_best_match[0]}, {geo_coords_best_match[1]}')
                        df.at[df_index[0], 'Predicted Location'] = str(f'{geo_coords_best_match[0]}, {geo_coords_best_match[1]}')
                        df.at[df_index[0], 'Issues'] = "Only one match found."

                else:
                    if df_index.any():
                        df.at[df_index[0], 'Issues'] = "Match not found."
                
            else:
                if df_index.any():
                    df.at[df_index[0], 'Issues'] = "GPS data is incomplete or not available in the image."

            df.to_csv('../results/NN-Eval.csv', index=False)
    


Working on files:   0%|          | 0/144 [00:00<?, ?it/s]

1/1 [==============================] - 1s 624ms/step


Working on files:   6%|▌         | 8/144 [00:28<08:02,  3.55s/it]

1/1 [==============================] - 1s 652ms/step


Working on files:   9%|▉         | 13/144 [00:52<09:06,  4.17s/it]

...

Working on files:  99%|█████████▊| 142/144 [1:43:37<02:11, 65.57s/it]

1/1 [==============================] - 1s 613ms/step


Working on files:  99%|█████████▉| 143/144 [1:44:00<00:52, 52.66s/it]

1/1 [==============================] - 1s 613ms/step


Working on files: 100%|██████████| 144/144 [1:44:56<00:00, 43.72s/it]


In [4]:
df = pd.read_csv('../results/NN-Eval.csv')
df

,Image,Location,Type,Notes,Best Match 1,Best Match 2,Predicted Location,Issues
0,iss065e012968,"23.742209, 120.791696",night,Taiwan,NaN,NaN,NaN,NaN
1,iss065e012854,"24.399968, 39.580929","city, desert","Medina, Saudi Arabia","24.243869682062538, 34.50965802339215","23.575597653538267, 31.96061662545327","23.909733667800403, 33.23513732442271",NaN
2,iss065e012122,"21.916841, 17.409515","mountains, desert","Tibesti Mountains, Chad","21.859068677152404, 9.468757021178105","22.869221341312457, 11.471575262415776","22.36414500923243, 10.470166141796941",NaN
3,iss040e092721,"37.901332, 15.333352","volcano, island",Mount Etna and Sicily,"37.99098385053961, 16.621670698969467","37.99098385053961, 14.983001228865902","37.99098385053961, 15.802335963917685",NaN
4,iss040e125148,"51.959667, 4.072454",port,Maasvlakte Rotterdam,"52.16862486467835, 4.377214208934959","51.264086859953494, 3.102693509965519","51.71635586231592, 3.739953859450239",NaN
...,...,...,...,...,...,...,...,...
139,iss056e126635,"22.511201, 32.948643",NaN,"Lake Nasser, Egypt","23.467633252740086, 34.45256236563611","23.299183131042493, 31.72144658213017","23.38340819189129, 33.08700447388314",NaN
140,iss036e049939,"-3.061136, 37.352760",volcano,"Mount Kilimanjaro, Tanzania","-1.4807118710086793, 35.69675412439839","-3.117553993689355, 36.42505166666665","-2.299132932349017, 36.06090289553252",NaN
141,iss057e114906,"33.245924, -119.475401",island,"San Nicolas Island, California",NaN,NaN,NaN,Match not found.
142,iss055e114459,"-17.620422, 139.094148",NaN,"Gulf of Carpentaria, Australia","-21.388845411417968, 139.35405069896947","-21.55756514109225, 137.35123245773178","-21.47320527625511, 138.35264157835064",NaN
